Script simple para usar el modelo de predicción de riesgo crediticio.

In [1]:
#Importo librerias necesarias.-

import joblib
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
#Funcion para cargar y definir la ruta el modelo.-

def cargar_modelo(ruta_modelo='modelos/modelo_riesgo_credito_random_forest.pkl'):
    """
    Carga el modelo de riesgo crediticio desde un archivo pickle
    
    Parámetros:
    -----------
    ruta_modelo : str
        Ruta al archivo del modelo .pkl
    
    Retorna:
    --------
    dict : Diccionario con el modelo y todos sus componentes
    """
    modelo_cargado = joblib.load(ruta_modelo)
    print(f"✓ Modelo cargado: {modelo_cargado['nombre_modelo']}")
    print(f"✓ Versión: {modelo_cargado['version']}")
    print(f"✓ Fecha entrenamiento: {modelo_cargado['fecha_entrenamiento']}")
    
    return modelo_cargado

In [3]:
#Funcion para predecir el riesgo crediticio.-

def predecir_riesgo(modelo_cargado, datos):
    """
    Realiza predicciones de riesgo crediticio para nuevos datos
    
    Parámetros:
    -----------
    modelo_cargado : dict
        Modelo cargado con cargar_modelo()
    datos : dict o DataFrame
        Datos del cliente. Puede ser un diccionario con un cliente
        o un DataFrame con múltiples clientes
    
    Retorna:
    --------
    DataFrame con las predicciones, probabilidades y nivel de riesgo
    """
    # Convertir a DataFrame si es diccionario
    if isinstance(datos, dict):
        datos = pd.DataFrame([datos])
    
    # Crear copia para no modificar el original
    datos_procesados = datos.copy()
    
    # 1. Imputar valores nulos
    if 'person_emp_length' in datos_procesados.columns:
        datos_procesados['person_emp_length'] = datos_procesados['person_emp_length'].fillna(
            modelo_cargado['mediana_emp_length']
        )
    if 'loan_int_rate' in datos_procesados.columns:
        datos_procesados['loan_int_rate'] = datos_procesados['loan_int_rate'].fillna(
            modelo_cargado['mediana_int_rate']
        )
    
    # 2. Encoding de variables categóricas
    for col, le in modelo_cargado['label_encoders'].items():
        if col in datos_procesados.columns:
            # Convertir a string y manejar valores nuevos
            datos_procesados[col] = datos_procesados[col].astype(str)
            datos_procesados[col] = datos_procesados[col].map(
                lambda x: x if x in le.classes_ else le.classes_[0]
            )
            datos_procesados[col] = le.transform(datos_procesados[col])
    
    # 3. Escalar si es necesario (solo para Regresión Logística)
    if modelo_cargado['requiere_escalado'] and modelo_cargado['scaler'] is not None:
        datos_procesados[modelo_cargado['columnas_numericas']] = \
            modelo_cargado['scaler'].transform(
                datos_procesados[modelo_cargado['columnas_numericas']]
            )
    
    # 4. Hacer predicciones
    predicciones = modelo_cargado['modelo'].predict(datos_procesados)
    probabilidades = modelo_cargado['modelo'].predict_proba(datos_procesados)[:, 1]
    
    # 5. Crear DataFrame con resultados
    resultados = pd.DataFrame({
        'Predicción': predicciones,
        'Probabilidad_Impago': probabilidades,
        'Riesgo': ['Alto' if p == 1 else 'Bajo' for p in predicciones],
        'Probabilidad_Pago': 1 - probabilidades
    })
    
    return resultados

In [4]:
# ============================================================================
# EJEMPLO DE USO
# ============================================================================

if __name__ == "__main__":
    # 1. Cargar el modelo
    print("="*70)
    print("CARGANDO MODELO DE RIESGO CREDITICIO")
    print("="*70)
    modelo = cargar_modelo()
    
    # 2. Ejemplo: Datos de un cliente
    print("\n" + "="*70)
    print("EJEMPLO DE PREDICCIÓN - CLIENTE INDIVIDUAL")
    print("="*70)
    
    cliente_ejemplo = {
        'person_age': 25,
        'person_income': 50000,
        'person_home_ownership': 'RENT',
        'person_emp_length': 3.0,
        'loan_intent': 'EDUCATION',
        'loan_grade': 'B',
        'loan_amnt': 10000,
        'loan_int_rate': 10.5,
        'loan_percent_income': 0.20,
        'cb_person_default_on_file': 'N',
        'cb_person_cred_hist_length': 5
    }
    
    resultado = predecir_riesgo(modelo, cliente_ejemplo)
    print("\n📊 Resultado de la predicción:")
    print(resultado.to_string(index=False))
    
    # 3. Ejemplo: Múltiples clientes
    print("\n" + "="*70)
    print("EJEMPLO DE PREDICCIÓN - MÚLTIPLES CLIENTES")
    print("="*70)
    
    clientes_multiples = pd.DataFrame({
        'person_age': [30, 45, 28],
        'person_income': [75000, 120000, 35000],
        'person_home_ownership': ['OWN', 'MORTGAGE', 'RENT'],
        'person_emp_length': [5.0, 15.0, 2.0],
        'loan_intent': ['PERSONAL', 'HOMEIMPROVEMENT', 'DEBTCONSOLIDATION'],
        'loan_grade': ['A', 'B', 'C'],
        'loan_amnt': [15000, 25000, 8000],
        'loan_int_rate': [8.5, 12.0, 15.5],
        'loan_percent_income': [0.20, 0.21, 0.23],
        'cb_person_default_on_file': ['N', 'N', 'Y'],
        'cb_person_cred_hist_length': [8, 12, 3]
    })
    
    resultados_multiples = predecir_riesgo(modelo, clientes_multiples)
    print("\n📊 Resultados de las predicciones:")
    print(resultados_multiples.to_string(index=False))
    
    print("\n" + "="*70)
    print("✓ Proceso completado exitosamente")
    print("="*70)

CARGANDO MODELO DE RIESGO CREDITICIO
✓ Modelo cargado: Random Forest
✓ Versión: 1.0
✓ Fecha entrenamiento: 2025-11-27 19:31:18

EJEMPLO DE PREDICCIÓN - CLIENTE INDIVIDUAL

📊 Resultado de la predicción:
 Predicción  Probabilidad_Impago Riesgo  Probabilidad_Pago
          0                 0.02   Bajo               0.98

EJEMPLO DE PREDICCIÓN - MÚLTIPLES CLIENTES

📊 Resultados de las predicciones:
 Predicción  Probabilidad_Impago Riesgo  Probabilidad_Pago
          0                 0.04   Bajo               0.96
          0                 0.04   Bajo               0.96
          0                 0.28   Bajo               0.72

✓ Proceso completado exitosamente
